In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
#Extract HTML tables into DataFrames
html_link = "https://en.wikipedia.org/wiki/List_of_best-selling_books"
best_selling=pd.read_html(html_link)

total_tables_found=len(best_selling)
print(f"There's a Total of {total_tables_found} tables found on the Wikipedia HTML")

There's a Total of 17 tables found on the Wikipedia HTML


In [3]:

print("The useful set of tables range from 1-5")

The useful set of tables range from 1-5


In [4]:
#Loop to retrieve and merge each Table(df)
best_selling_df=best_selling[1]

for i in range(2,6):
    best_selling_df=best_selling_df.append(best_selling[i],ignore_index=True)
    
best_selling_df


,Book,Author(s),Original language,First published,Approximate sales,Genre
0,A Tale of Two Cities,Charles Dickens,English,1859,200 million[15],Historical fiction
1,The Little Prince (Le Petit Prince),Antoine de Saint-Exupéry,French,1943,150 million[16][17][18][19][20][21],fantasy (picture book)
2,Harry Potter and the Philosopher's Stone,J. K. Rowling,English,1997,120 million[9][22],"fantasy, mystery"
3,The Hobbit,J. R. R. Tolkien,English,1937,100+ million [15],fantasy
4,And Then There Were None,Agatha Christie,English,1939,100 million[15],mystery
...,...,...,...,...,...,...
160,The Front Runner,Patricia Nell Warren,English,1974,10 million[169],NaN
161,The Goal,Eliyahu M. Goldratt,English,1984,10 million[170],NaN
162,Fahrenheit 451,Ray Bradbury,English,1953,10 million[171],NaN
163,Angela's Ashes,Frank McCourt,English,1996,10 million[172],NaN


In [5]:
# #Clean Approximate sales
best_selling_df['Approximate sales']= best_selling_df['Approximate sales'].str.split(' million').str[0]
best_selling_df['Approximate sales']= best_selling_df['Approximate sales'].str.split('–').str[0]
best_selling_df['Approximate sales']= best_selling_df['Approximate sales'].str.split('+').str[0]

best_selling_df.at[best_selling_df.index[best_selling_df['Approximate sales']=='>80'],['Approximate sales']]= 80

best_selling_df

,Book,Author(s),Original language,First published,Approximate sales,Genre
164,The Story of My Experiments with Truth (સત્યના...,Mohandas Karamchand Gandhi,Gujarati,1925-1929,10,NaN


In [6]:
# Cleaning First published
best_selling_df['First published']=best_selling_df['First published'].astype('str')
best_selling_df.at[best_selling_df.index[best_selling_df['First published'] =='1925-1929'],['First published']]= '1925'


best_selling_df['First published'].unique()

array(['1859', '1943', '1997', '1937', '1939', '1791', '1950', '1887',
       '1881', '2003', '1998', '1999', '2000', '2005', '2007', '1988',
       '1951', '1992', '1880', '1984', '1967', '1955', '1946', '1908',
       '1877', '1980', '1975', '1972', '1976', '1952', '1902', '1970',
       '1969', '1899', '1960', '1979', '1991', '1932', '1869', '1947',
       '1977', '2002', '1966', '1925', '1936', '1938', '1949', '2009',
       '2008', '1945', '1788', '1934', '1989', '1935', '1993', '2012',
       '2010', '1957', '2015', '1929', '1974', '1958', '2004', '1971',
       '1986', '1885', '1982', '1813', '1923', '1963', '2006', '1973',
       '1965', '1964', '1968', '1981', '1995', '2011', '1994', '1942',
       '1985', '2001', '1933', '1962', '1987', '1956', '1948', '1304',
       '2018', '1961', '1978', '1953', '1996'], dtype=object)

In [7]:
#Fix Book names where there are two titles(English and Original language)
best_selling_df['Book']= best_selling_df['Book'].str.split('(').str[0]
best_selling_df

,Book,Author(s),Original language,First published,Approximate sales,Genre
164,The Story of My Experiments with Truth,Mohandas Karamchand Gandhi,Gujarati,1925,10,NaN


In [8]:
#Fix Author "E.B. White; illustrated by Garth Williams"
best_selling_df.at[best_selling_df.index[best_selling_df['Author(s)'] =='E.B. White; illustrated by Garth Williams'],['Author(s)']]= 'E.B. White'

In [9]:
# Check for repeatedness on Languages
best_selling_df['Original language'].unique()

array(['English', 'French', 'Chinese', 'Italian', 'Portuguese', 'Spanish',
       'Swiss German', 'Norwegian', 'Russian', 'Dutch', 'Swedish',
       'Japanese', 'Hindi', 'German', 'Czech', 'Yiddish', 'Gujarati'],
      dtype=object)

In [10]:
#Change Swiss German language to German
best_selling_df.at[best_selling_df.index[best_selling_df['Original language'] =='Swiss German'],['Original language']]= 'German'
best_selling_df['Original language'].unique()


array(['English', 'French', 'Chinese', 'Italian', 'Portuguese', 'Spanish',
       'German', 'Norwegian', 'Russian', 'Dutch', 'Swedish', 'Japanese',
       'Hindi', 'Czech', 'Yiddish', 'Gujarati'], dtype=object)

In [11]:
# Change column names to match SQL column titles
best_selling_df=best_selling_df.rename(columns={'Book':'title',
                               'Author(s)':'authors',
                               'Original language':'language_code',
                               'First published':'first published',
                               'Approximate sales':'approximate sales',
                               'Genre':'genre'})



In [12]:
# Check for duplicates on Book titles
duplicated_books=best_selling_df[best_selling_df.duplicated(['title'])]
print(duplicated_books)

Empty DataFrame
Columns: [title, authors, language_code, first published, approximate sales, genre]
Index: []


In [13]:
# Export to csv
best_selling_df.to_csv('best_selling_books.csv')

PermissionError: [Errno 13] Permission denied: 'best_selling_books.csv'